In [ ]:
#importing packages
import re
import cv2
import plotly
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, VBox
from PIL import Image
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from mpl_toolkits import mplot3d

#import helper functions to be used throughout
from piezo_tension_sensitivity_helper_functions import *

### Initializing dictionary and restarting pressure step

The pressure/tension data will be collected in the following dictionary. These cells only need to be run at the start of each new patch or if an image for a pressure step needs to be rerun. Othersie, leave the "pressure_step" at zero so your analysis starts with the first image for the patch.

In [ ]:
#create dictionary of pressures and tension to upload to a dataframe later
pressure_tension = {}

In [ ]:
#getting the croped version of the image along with the coordinates of the four corners with respect to the original image
pressure_step =0

### Loading images

This is the first cell used for each image. Best practice would be to have images that contain the pressure step value followed by mmHg. This way when an image for a patch is completed, the impage for the next pressure step will be automatically ready to upload (see below). For example, naming images like such: date+'_pz2_tension_'+patch_num+'_'+str(pressure_step)+'mmHg.jpg'

One loaded, a separate window containing the image will appear. Click and drag a boudning box around the membrane done and immediate pipette walls to determine an ROI. Make sure to have a sufficient number of piexls ablve and below and most extremem regions of the membrane as this will later become important for membrane fitting.

The selected ROI will appear afterwards.

In [ ]:
date='xxx'
patch_num='xx'

image = date+'_pz2_tension_'+patch_num+'_'+str(pressure_step)+'mmHg.jpg'
file = 'file path/'+image

roi_cropped, cropped_coordinates, img_raw = get_cropped_image(file)

### Determine initial membrane location

The first function (find_mambrane) will determine the vertical location of the darkest agerave for groups of five pixels, which will include the membrane and the pipette walls. It may also inlcude pooints that are clearly not the membrane which will need to me removed manually.

The second function (fix_mem_loc) will be used to isolate the pixels that ar epart of the membrane. Three input boxes will appear:
- membrane start (left x coordinate for where the membrane makes contatc with the pipette wall)
 
 
- membrane end: (right x coordinate for where the membrane makes contatc with the pipette wall)
 
 
- Problem points (x val) / n: if there are not points between "membrane start" and "membrane end" type n. If there are, the x coordinates for each point need to be entered manually, separated by a comma

In [ ]:
#generating an array with the locations of darkest areas (per 3 pixels)
mem_loc_df = find_membrane(roi_cropped)

In [ ]:
#cutting the membrane coordinate dataframe to exclude the pipette walls or rogue points
mem_loc_fixed = fix_mem_loc(mem_loc_df)

### Determining precise membrane location

The next function (gaus_find_membrane) uses a gaussian fit at each point to more accurately determine the location of the membrane. Using the initial membrane locations from before, 12 pixels above and below are fit with a gaussian and the center location is used as the new "true" location of the membrane. Additionally, the sigma for each gaussia fit is also recorded and will be used later for circular fitting. 

In [ ]:
#find membrane using gaussian fits
df_gaus_data, gaus_points_fig = gaus_find_membrane(mem_loc_fixed, roi_cropped)

Sometimes the gaussian fitting does not work and erroneous points arise. The next function (remove_low_gaus_sigma) will highlight and remove those points automatically. The last function (fix_mem_loc_gaus) will allow you to make sure these points are removed. If they are and everything looks fine, typs "n" into the input box. If points do remain, same as before, input their individual x coordinates, separated by commas, into the input box to remove them.

In [ ]:
#remove data with sigma<1 (from bad gaussian fits)
df_gaus_data=remove_low_gaus_sigma(df_gaus_data, roi_cropped)

In [ ]:
#used if gaussian fit has a misfit with a sigma that pulls the circle fit way off
df_gaus_data = fix_mem_loc_gaus(df_gaus_data)

### Determining membrane curvature and tension

The last step for image analysis is to fit the membrane locations with a circular fit to determine membrane curvature and tension. The next function (fit_circle_gaus_weighted) uses the gaussian determined membrane points and fits them with a circle. The sigmas for each point are used to weight the fit.

In [ ]:
#perform a circle fit on our membrane data
circle_fit_data_popt, fit_stdvs = fit_circle_gaus_weighted(circle_fit, df_gaus_data, roi_cropped)

The next funciton (combined_image_traces) will overlay the circular fit with the original image for visual confocmation. Additionally, the values for the theoretical radius and calculated membrane tension are printed below.

In [ ]:
#generate tension and overlay circle fit on the OG image
overlay_image, tension, radius = combined_image_traces(circle_fit_data_popt, cropped_coordinates ,df_gaus_data, img_raw, pressure_step)

overlay_image.savefig('Z:/All_Staff/Grandl Lab/Michael Sindoni/misc_old/neurobiology_retreat/retreat_2025/figures/svgs/mem_in_pip_example.svg', transparent=True, dpi=1200)


### Update dictionary

The next function (update_tension_pressure) will update the dictionary containing your data. The "pressure_step" variable will also increase whcih is why having images with a pressure step label will come in handy. This way, when I scroll back and upload a new image, it will be the next image generated for that patch.

In [ ]:
#updates the pressure/tension
updated_presure_tension_dict = update_tension_pressure(pressure_tension, tension, pressure_step, circle_fit_data_popt, fit_stdvs)

pressure_step +=5

### Export data

The cell below will create a dataframe for all data and will copy it to your clipboard to be pasted into an excel file.

In [ ]:
df_tension_final = generate_final_tension_df(updated_presure_tension_dict)
df_tension_final.to_clipboard(excel=True, index=False, header=None)
df_tension_final